In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.interpolate import griddata
from astropy.time import Time

## Define functions used

In [ ]:
# Read in each data scan at selected frequency index
def read_file(dir_in,filename,f_idx):

    dva_file = fits.open(dir_in+filename)

    t = np.empty(len(dva_file[1].data))
    az = np.empty(len(dva_file[1].data))
    el = np.empty(len(dva_file[1].data))
    RA = np.empty(len(dva_file[1].data))
    dec = np.empty(len(dva_file[1].data))
    StokesI = np.empty(len(dva_file[1].data))
    StokesQ = np.empty(len(dva_file[1].data))
    StokesU = np.empty(len(dva_file[1].data))
    date_mjd = np.empty(len(dva_file[1].data))
    
    for i in range(0,len(t)):
        date = dva_file[1].data[i][2]+'T00:00:00.0'
        date_mjd[i] = Time(date, format='isot',scale='utc').mjd
        az[i] = dva_file[1].data[i][13]
        el[i] = dva_file[1].data[i][14]
        RA[i] = dva_file[1].data[i][11]
        dec[i] = dva_file[1].data[i][12]
        t[i] = dva_file[1].data[i][7]
        StokesI[i] = dva_file[1].data[i][8][0][0][0][f_idx]
        StokesQ[i] = dva_file[1].data[i][8][0][0][1][f_idx]
        StokesU[i] = dva_file[1].data[i][8][0][0][2][f_idx] 

    return az, el, RA, dec, t, StokesI, StokesQ, StokesU, date_mjd

## Establish directories and frequencies

In [ ]:
####################################################################
#  Pick the scan directory and whether we want East or West scans
####################################################################
freqrange = 'second' # 'second'=higher frequencies, 'first'=lower frequencies
elrange = 'low'      # replace with 'high' for high elevation scans
EW = 'W'
####################################################################

scan_list = 'scan_list_'+elrange+'el.txt'
dir_in = '/srv/data/dva/New_mini_survey_'+freqrange+'_half_'+elrange+'/'

freqs = []
with open(dir_in+'ListOfFrequencies.txt') as f:
    for line in f:
        freqs.append(float(line.split()[0]))

# Use this printout of the frequencies to chose which channel to read in:
for i in range(0,len(freqs)):
    print(i,freqs[i])

## Pick the frequency:

In [ ]:
###################################
f_idx = 8
###################################

## Read in data for selected frequency and set of scans

In [ ]:
print('Reading in files for f = '+str(freqs[f_idx])+' MHz...')
print('----------------------------------------')
print('')
az_all = []
el_all = []
RA_all = []
dec_all = []
t_all = []
I = []
Q = []
U = []
date_mjd_all = []

i = 0
with open(dir_in+scan_list) as f:
    for line in f:
        if (line[26] == EW):
            print(i+1,line.split()[0])
            az,el,RA,dec,t,StokesI,StokesQ,StokesU,date_mjd = read_file(dir_in,line.split()[0],f_idx)
            az_all.append(az)
            el_all.append(el)
            RA_all.append(RA)
            dec_all.append(dec)
            t_all.append(t)
            I.append(StokesI)
            Q.append(StokesQ)
            U.append(StokesU)
            date_mjd_all.append(date_mjd)
            i = i+1

## Turn the lists into arrays (everything is 1D at this point - scan files were just stitched together end to end)

In [ ]:
az_arr = np.concatenate(az_all)
el_arr = np.concatenate(el_all)
RA_arr = np.concatenate(RA_all)*360/24 # Convert from hours to degrees
dec_arr = np.concatenate(dec_all)
I_arr = np.concatenate(I)
Q_arr = np.concatenate(Q)
U_arr = np.concatenate(U)
date_mjd_arr = np.concatenate(date_mjd_all)

print(Q_arr.shape)

## Median-subtract Q and U (to approximate instrumental polarization offset)

In [ ]:
Q_arr_ms = Q_arr-np.nanmedian(Q_arr)
U_arr_ms = U_arr-np.nanmedian(U_arr)

## Pick a grid spacing size (in degrees)

In [ ]:
#############################
dxy = 0.5
#############################
numRA = int(360./dxy)
numdec= int(180./dxy)

print(numRA,numdec)

## Interpolate onto a grid (this may take a few minutes) 

In [ ]:
# The original coordinates for the datapoint positions
points = tuple([RA_arr,dec_arr])

# A regular grid in RA/dec:
ra_reg = np.linspace(0+dxy/2,360-dxy/2,numRA)
dec_reg = np.linspace(-90+dxy/2,90-dxy/2,numdec)
ra_2D, dec_2D = np.meshgrid(ra_reg, dec_reg)

# Flatten the 2D grid in order to match with 1D data format
reg_grid = tuple([ra_2D.flatten(),dec_2D.flatten()])

# Print out sizes of various arrays and tuples to check
print(type(points))
print(len(points))
print(points[0].shape)
print(Q_arr.shape)
print(type(reg_grid))
print(len(reg_grid))
print(reg_grid[0].shape)
print('')

# Do the interpolation (note: interpolated data are still 1D array at this point)
Q_interp_1D = griddata(points,Q_arr_ms,reg_grid,method='linear')
U_interp_1D = griddata(points,U_arr_ms,reg_grid,method='linear')

print(Q_interp_1D.shape)

## Now turn the interpolated data into 2D format

In [ ]:
Q_interp_2D = np.reshape(Q_interp_1D,ra_2D.shape)[:,::-1]
U_interp_2D = np.reshape(U_interp_1D,ra_2D.shape)[:,::-1]

print(Q_interp_2D.shape)

## Plot interpolated and original scans (plotting Q for now)

In [ ]:
fig,ax = plt.subplots(2,1,figsize=(20,10)) 

# Use imshow for interpolated data set:
im1 = ax[0].imshow(Q_interp_2D,vmin=-15,vmax=15,cmap='RdBu_r',origin='lower',
                  extent=[360,0,-90,90])
# Use scatterplot for individual scans, since points not on regular grid:
im2 = ax[1].scatter(RA_arr,dec_arr,s=0.1,c=Q_arr_ms,vmin=-15,vmax=15,cmap='RdBu_r')

divider = make_axes_locatable(ax[0])
cax = divider.append_axes("right", size="1%", pad=0.05)
plt.colorbar(im1, cax=cax)

divider = make_axes_locatable(ax[1])
cax = divider.append_axes("right", size="1%", pad=0.05)
plt.colorbar(im2, cax=cax)

ax[0].set_title('Interpolated')
ax[1].set_title('Original scans')

for i in range(0,2):
    ax[i].set_aspect('equal') 
    ax[i].set_xlabel('RA')
    ax[i].set_ylabel('dec')
    ax[i].set_xlim(360,0)
    ax[i].set_ylim(-25,65)

## Create a 2D plate-caree FITS header for writing out files

In [ ]:
hdr_car_2D = fits.Header.fromstring("""
NAXIS   =                    2
CUNIT1  = 'deg     '
CUNIT2  = 'deg     '
""", sep='\n')

hdr_car_2D['COORDSYS']  = 'icrs'
hdr_car_2D['NAXIS1']  = Q_interp_2D.shape[1] 
hdr_car_2D['NAXIS2']  = Q_interp_2D.shape[0]

hdr_car_2D['CTYPE1']  = 'RA---CAR'
hdr_car_2D['CRPIX1']  = Q_interp_2D.shape[1]/2.+0.5 
hdr_car_2D['CRVAL1']  = 360./2.          
hdr_car_2D['CDELT1']  = -dxy

hdr_car_2D['CTYPE2']  = 'DEC--CAR'
hdr_car_2D['CRPIX2']  = Q_interp_2D.shape[0]/2.+0.5
hdr_car_2D['CRVAL2']  = 0.
hdr_car_2D['CDELT2']  = dxy

print(repr(hdr_car_2D))

## Write out files

In [ ]:
###########################################################
# Pick filename and output directory here:
outfilename = 'test.fits' # Pick better names! :D 
outfiledir = '/srv/data/dva/dva_map_test_results/George/'
overwrite = True # CAREFUL! If you don't want to accidentally
                 # overwrite files, set this to False
###########################################################

fits.writeto(outfiledir+outfilename,Q_interp_2D,header=hdr_car_2D,
             overwrite=overwrite,output_verify='fix')
